In [1]:
import pandas as pd
import json 
import os
from bs4 import BeautifulSoup
import requests
import re
import time

def get_config(home_dir):
    path = "{}/config.json".format(home_dir)

    with open(path) as f:
        config = f.readlines()[0]
    
    config_dict = json.loads(config)

    return config_dict

def get_html(url):
    wait_time = 1
    html = requests.get(url, verify=False).content.decode("utf-8") 
    soup = BeautifulSoup(re.sub("<!--|-->","", html), "lxml")  
    time.sleep(wait_time)
    return soup

config_dir = os.getcwd().replace('/notebooks', '')
config = get_config(config_dir)            

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class Season():
    def __init__(self, year):
        self.season = year
        config_dir = os.getcwd().replace('/notebooks', '')
        config = get_config(config_dir)        
        self.base_url = config['base_url']
        
        self.schedule_url = "{}/years/{}/games.htm".format(self.base_url, self.season)
        self.schedule_games_id = "games"
        self.schedule_page = self.get_season_schedule()
    
    def get_season_schedule(self):
        
        url = self.schedule_url
        schedule_html = requests.get(url, verify=False).content.decode("utf-8") 
        schedule_soup = BeautifulSoup(re.sub("<!--|-->","", schedule_html), "lxml")  
        
        return schedule_soup
    def get_schedule_table(self):
        page = self.schedule_page
        tbl_id = self.schedule_games_id
        schedule_table = page.find(id=tbl_id)
        
        return schedule_table
    
    def get_weekly_dict(self, games_table):
        
        all_trs = games_table.findAll('tr')
        
        game_rows = list(filter(lambda x: x.has_attr('class') == False, all_trs))
        weekly_dict = {}
        
        for game in game_rows:
            week_num = game.find("th", {"data-stat" : "week_num"}).string 

            if week_num in ['Week', None]:
                continue
            if week_num not in weekly_dict.keys():
                weekly_dict[week_num] = [game]
            else:
                weekly_dict[week_num].append(game)
        return weekly_dict
season = Season(2016)

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
games_table = season.get_schedule_table()
weekly_dict = season.get_weekly_dict(games_table)

    #print(game)
        #weekly_dict[week_num] = weekly_dict[week_num].append(game)
#weekly_dict = season.get_weekly_dict(games_table)
#print(games_rows[1])

# for game in games_rows[0:5]:
#     print("---")
#     print(game)

In [4]:
# for week, games in weekly_dict.items():
    
#     print("{}: {} games".format(week, len(games)))

games = weekly_dict["5"]
print(games[0])

<tr><th class="right " csk="5" data-stat="week_num" scope="row">5</th><td class="left " data-stat="game_day_of_week">Thu</td><td class="left " csk="2016-10-06" data-stat="game_date">October 6</td><td class="right " data-stat="gametime">8:27PM</td><td class="left " data-stat="winner"><strong><a href="/teams/crd/2016.htm">Arizona Cardinals</a></strong></td><td class="right " data-stat="game_location">@</td><td class="left " data-stat="loser"><a href="/teams/sfo/2016.htm">San Francisco 49ers</a></td><td class="center " data-stat="boxscore_word"><a href="/boxscores/201610060sfo.htm">boxscore</a></td><td class="right " data-stat="pts_win"><strong>33</strong></td><td class="right " data-stat="pts_lose">21</td><td class="right " data-stat="yards_win">288</td><td class="right " data-stat="to_win">0</td><td class="right " data-stat="yards_lose">286</td><td class="right " data-stat="to_lose">3</td></tr>


In [5]:
class Week(Season):
    def __init__(self, year, week, games, upcoming = False):
        self.season = year       
        self.base_url = config['base_url']
        self.games = games
        url_week_dict = {
            "WildCard" : 18,
            "Divisional" : 19,
            "ConfChamp" : 20,
            "SuperBowl" : 21
        }
        url_week = url_week_dict[week] if week in url_week_dict.keys() else int(week)
        self.week_url = "{}/years/{}/week_{}.htm".format(self.base_url, self.season, url_week)    
        

In [22]:
week = Week(2016, 5, games)
week.season
week.

2016

In [7]:
class Game():
    def __init__(self, year, week, game_summary_row, upcoming=False):
        
        # set these self variables first so all other fxns in init can run
        self.season = year
        self.week = week
        self.upcoming = upcoming
        self.base_url = config['base_url']
        self.game_url_append = game_summary_row.find("td", {"data-stat":"boxscore_word"}).a["href"]
        self.game_full_url = self.base_url + self.game_url_append        
        self.home_winner_col_name = "home_winner"
        self.game_link_col_name = "game_link"
        self.game_summary_row = game_summary_row
        self.game_summary_cols = game_summary_row.findAll('td')        
        self.game_summary_output_cols = [
            "year",
            "week_num",
            "game_date",
            self.game_link_col_name,
            self.home_winner_col_name, 
            "winner",
            "pts_win",
            "loser",
            "pts_lose"
        ]
        # end
        
        self.game_summary_dict = self.clean_game_summary(game_summary_row)  
        self.teams_list = self.get_teams(game_summary_row)
        self.game_html_page = self.get_game_page()
              
        self.favorite = self.get_favorite_or_underdog(favorite_flg = True)
        self.underdog = self.get_favorite_or_underdog(favorite_flg = False)
        self.home_team = self.get_home_or_vis(home_flg = True)
        self.vis_team = self.get_home_or_vis(home_flg = False)
        self.line = self.get_vegas_stats(stat = "line")
        self.over_under = self.get_vegas_stats(stat = "over_under")
        
    def get_home_or_vis(self, home_flg):
        
        game_summary_dict = self.game_summary_dict
        
        if self.upcoming == False:
            
            winner = game_summary_dict['winner']
            loser = game_summary_dict['loser']
            home_winner = game_summary_dict['home_winner']
            
            if home_flg == True:
                team =  winner if home_winner == True else loser
            else:
                team = loser if home_winner == True else winner 
                
        else:
            team = None
        
        return team
    
    def get_teams(self, game_summary_row):
        
        game_summary_dict = self.game_summary_dict
        teams_list = []
        
        for elem in ["winner", "loser"]:
            team = {
                "team" : game_summary_row.find('td', {'data-stat' : elem}).a.string, 
                "link" : game_summary_row.find('td', {'data-stat' : elem}).a['href']
            }
            
            if elem == "winner" and game_summary_dict['home_winner'] == True:
                team["home_flg"] = True 
            elif elem == "winner" and game_summary_dict['home_winner'] == False:
                team["home_flg"] = False
            elif elem == "loser" and game_summary_dict['home_winner'] == True:
                team["home_flg"] = False
            elif elem == "loser" and game_summary_dict['home_winner'] == False:
                team["home_flg"] = True
            teams_list.append(team)
        
        return teams_list
    
    def get_game_page(self):
        
        game_soup = get_html(self.game_full_url)
        return game_soup
    
    def get_vegas_stats(self, stat):
        
        game_html_page = self.game_html_page
        
        game_info_tbl = game_html_page.find("table", {"id":"game_info"})
        
        if stat == 'line':
            
            vegas_pattern = re.compile(r'Vegas Line')
            line_string = game_html_page.find('th', text=vegas_pattern).parent.td.string
            line_string_split = re.split('([\+-])', line_string)        
            return_val = float(line_string_split[-1])
            
        elif stat == "over_under":
            vegas_pattern = re.compile(r'Over/Under')
            ou_string = game_html_page.find('th', text=vegas_pattern).parent.td.contents[0]
            return_val = float(ou_string.strip())
            
        return return_val
    
    def get_favorite_or_underdog(self, favorite_flg):
        
        upcoming = self.upcoming
        game_page = self.game_html_page
        teams = [x["team"] for x in self.teams_list]
        game_info_tbl = game_page.find("table", {"id":"game_info"})
        vegas_pattern = re.compile(r'Vegas Line')
        line_string = game_info_tbl.find('th', text=vegas_pattern).parent.td.string
        line_string_split = re.split('([\+-])', line_string)
        sign = line_string_split[1]
        if favorite_flg == True and sign == "-":
            team = line_string_split[0].strip()
        elif favorite_flg == True and sign == "+":
            teams.remove(line_string_split[0].strip())
            team = teams[0]
        elif favorite_flg == False and sign == "-":
            teams.remove(line_string_split[0].strip())
            team = teams[0]
        elif favorite_flg == False and sign == "+":
            team = line_string_split[0].strip()
        
        #print(game_info_tbl)
#         if upcoming == False:
#             team = self.game_summary_dict['winner'] if 
        return team
    
    def clean_game_summary(self, game_summary_row):
        
        game_results_cols = self.game_summary_output_cols
        game_summary_cols = game_summary_row.findAll('td')
        
        game_summary_dict = {}
        
        for td in game_summary_cols:
            game_summary_dict = self.clean_game_summary_col(game_summary_dict, td)
        return game_summary_dict
    
    def clean_game_summary_col(self, game_summary_dict, td):
        
        data_stat = td.get('data-stat')
        if data_stat in ["game_date", "pts_lose"]:
            game_summary_dict[data_stat] = td.string
        if data_stat in ["winner", "loser"]:
            game_summary_dict[data_stat] = td.a.string
        if data_stat == "game_location":
            game_summary_dict[self.home_winner_col_name] = False if td.string == "@" else True
        if data_stat == "pts_win":
            game_summary_dict[data_stat] = td.strong.string

        return game_summary_dict

In [8]:
game = Game(2016, 5, week.games[0])

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
print(game.game_url_append)
print(game.teams_list)

/boxscores/201610060sfo.htm
[{'team': 'Arizona Cardinals', 'link': '/teams/crd/2016.htm', 'home_flg': False}, {'team': 'San Francisco 49ers', 'link': '/teams/sfo/2016.htm', 'home_flg': True}]


In [10]:
class Team(Game):
    def __init__(self, year, week, game_html, team_name, opponent, home_flg):
        
        self.team_table_id = "home" if home_flg == True else "vis"
        self.opp_table_id =  "vis" if home_flg == True else "home"
        self.home_flg = home_flg
        self.team = team_name
        self.opponent = opponent
        self.game_html = game_html
        self.season = year
        self.week = week
        self.team_abbrev = self.get_team_abbrev(opponent_flg = False)
        self.opp_abbrev = self.get_team_abbrev(opponent_flg = True)
        
    def get_team_abbrev(self, opponent_flg):
        
        team = self.team if opponent_flg == False else self.opponent
        
        team_abbrev_dict = {
            "Arizona Cardinals" : "ARI",
            "San Francisco 49ers" : "SFO"
        }
        
        abbrev = team_abbrev_dict[team]
        return abbrev
        
        
    def get_indiv_game_stats(self):
        game_stats_id = "team_stats"
        game_html = team.game_html
        stats_table = game_html.find(id=game_stats_id).tbody
        stats_table_rows = stats_table.findAll('tr')

        stats_table_dict = {}       
        
        for tr in stats_table_rows:
            stats_table_dict = self.clean_stats_table_row(tr, stats_table_dict)
    
        return stats_table_dict
    
    def clean_stats_table_row(self, row, stats_table_dict):
        stat_name_dict = {
            "First Downs" : "first_downs",
            "Rush" : "rush_att",
            "Yds" : "rush_yds",
            "TDs" : "rush_tds",
            "Cmp" : "pass_comp",
            "Att" : "pass_att",
            "Yd" : "pass_yds",
            "TD" : "pass_tds",
            "INT" : "pass_int",
            "Sacked" : "sacks",
            "Net Pass Yards" : "net_pass_yds",
            "Total Yards" : "total_yds",
            "Fumbles" : "fumbles",
            "Lost" : "fumbles_lost",
            "Turnovers" : "turnovers",
            "Penalties" : "penalties",
            "Yards" : "penalty_yds",
            "Third Down Conv." : "third_down_conv",
            "Fourth Down Conv."  : "fourth_down_conv",
            "Time of Possession" : "time_of_poss"
        }    
        team_id = "{}_stat".format(self.team_table_id)
        opp_id = "{}_stat".format(self.opp_table_id)
        stat = row.th.string
        team_stat = row.find('td', {"data-stat" : team_id}).string
        opp_stat = row.find('td', {"data-stat" : opp_id}).string
        
        if "-" in stat:
            stat_name_split = stat.split("-")
            team_stat_split = team_stat.split("-")
            opp_stat_split = opp_stat.split("-")

            for idx, stat in enumerate(stat_name_split):
                stdized_stat = stat_name_dict[stat]
                team_stat = team_stat_split[idx]
                opp_stat = opp_stat_split[idx]
                stats_table_dict["team_{}".format(stdized_stat)] = team_stat
                stats_table_dict["opp_{}".format(stdized_stat)] = opp_stat
        else:
            stdized_stat = stat_name_dict[stat]        
            stats_table_dict["team_{}".format(stdized_stat)] = team_stat
            stats_table_dict["opp_{}".format(stdized_stat)] = opp_stat

        return stats_table_dict

In [25]:
arz = game.teams_list[0]
opp = game.teams_list[1]
# for idx, team in enumerate(game.teams_list):
    
#     team = game.teams_list[idx]
#     opp = game.teams_list[abs(idx - 1)]
#     print("{} vs {}".format(team["team"], opp["team"]))
print(game.favorite)
print(game.game_summary_dict)
print(game.home_team)
team = Team(game.season, game.week, game.game_html_page, arz["team"], opp["team"], arz["home_flg"])



Arizona Cardinals
{'game_date': 'October 6', 'winner': 'Arizona Cardinals', 'home_winner': False, 'loser': 'San Francisco 49ers', 'pts_win': '33', 'pts_lose': '21'}
San Francisco 49ers


In [12]:
def accumulate_rush_dir_stats(rush_dir_dict, tr_list, data_stat, opponent_flg):
    
    vals = [tr.find("td", {"data-stat": data_stat}).string for tr in tr_list]
    vals = [int(val) if val is not None else 0 for val in vals]
    rush_dir_sum = sum(vals)
    
    dict_key = "opp_{}".format(data_stat) if opponent_flg == True else "team_{}".format(data_stat)
    rush_dir_dict[dict_key] = rush_dir_sum
    
    return rush_dir_dict

def filter_rush_dir_rows(row, abbrev):
    if row.find("td", {"data-stat" : "team"}) is None:
        val = False
    elif row.find("td", {"data-stat" : "team"}).string == abbrev:
        val = True
    else:
        val = False
    
    return val


In [13]:
print(team.team_abbrev)
print(team.opp_abbrev)
from itertools import accumulate 

game_html = team.game_html
rush_dir_tbl = game_html.find("table", {"id" : "rush_directions"}).tbody

rush_dir_rows = rush_dir_tbl.findAll("tr")

team_rush_dir_rows = list(filter(lambda x: filter_rush_dir_rows(x, team.team_abbrev), rush_dir_rows))
opp_rush_dir_rows = list(filter(lambda x: filter_rush_dir_rows(x, team.opp_abbrev), rush_dir_rows))

rush_dirs = ["le", "lt", "lg", "md", "rg", "rt", "re"]
rush_dir_stats = ["rush", "rush_yds", "rush_td"]

rush_dir_dict = {}
for direction in rush_dirs:
    for rush_stat in rush_dir_stats:
        data_stat = "{}_{}".format(rush_stat, direction)
        rush_dir_dict = accumulate_rush_dir_stats(rush_dir_dict, team_rush_dir_rows, data_stat, opponent_flg = False)
        rush_dir_dict = accumulate_rush_dir_stats(rush_dir_dict, opp_rush_dir_rows, data_stat, opponent_flg = True)
                
#         print(data_stat)
#         team_vals = [tr.find("td", {"data-stat": data_stat}).string for tr in team_rush_dir_rows]
#         team_vals = [int(val) if val is not None else 0 for val in team_vals]
#         team_sum = sum(team_vals)
        
#         opp_vals = [tr.find("td", {"data-stat": data_stat}).string for tr in opp_rush_dir_rows]
#         opp_vals = [int(val) if val is not None else 0 for val in opp_vals]
#         opp_sum = sum(opp_vals)
#         print(team_sum, opp_sum)
print(rush_dir_dict)        

#player_stats = list(filter(lambda x: x["data-stat"] in game_stats_list, player_stats))
# stats_tbl = team.get_indiv_game_stats()
# print(game.game_full_url)
# print(stats_tbl)

ARI
SFO
{'team_rush_le': 1, 'opp_rush_le': 3, 'team_rush_yds_le': 11, 'opp_rush_yds_le': 12, 'team_rush_td_le': 0, 'opp_rush_td_le': 0, 'team_rush_lt': 7, 'opp_rush_lt': 2, 'team_rush_yds_lt': 44, 'opp_rush_yds_lt': 4, 'team_rush_td_lt': 0, 'opp_rush_td_lt': 0, 'team_rush_lg': 7, 'opp_rush_lg': 6, 'team_rush_yds_lg': 20, 'opp_rush_yds_lg': 11, 'team_rush_td_lg': 0, 'opp_rush_td_lg': 0, 'team_rush_md': 6, 'opp_rush_md': 13, 'team_rush_yds_md': 8, 'opp_rush_yds_md': 48, 'team_rush_td_md': 0, 'opp_rush_td_md': 1, 'team_rush_rg': 7, 'opp_rush_rg': 6, 'team_rush_yds_rg': 10, 'opp_rush_yds_rg': 44, 'team_rush_td_rg': 1, 'opp_rush_td_rg': 0, 'team_rush_rt': 6, 'opp_rush_rt': 0, 'team_rush_yds_rt': 77, 'opp_rush_yds_rt': 0, 'team_rush_td_rt': 0, 'opp_rush_td_rt': 0, 'team_rush_re': 1, 'opp_rush_re': 3, 'team_rush_yds_re': 4, 'opp_rush_yds_re': 8, 'team_rush_td_re': 1, 'opp_rush_td_re': 1}


In [14]:
html = team.game_html
team_snaps_tbl = team.game_html.find("table", {"id": "{}_snap_counts".format(team.team_table_id)})
team_snaps_tbl_body = team_snaps_tbl.tbody
team_snaps_rows = team_snaps_tbl_body.findAll('tr')
print(team.season, team.week)

2016 5


In [15]:
game_pass_stats = [
    "pass_cmp", "pass_att", "pass_yds", "pass_td" \
    "pass_td", "pass_int", "pass_sacked", "pass_sacked_yds" \
    "pass_long", "pass_rating", "fumbles", "fumbles_lost"
]

game_rush_stats = [
    "rush_att", "rush_yds", "rush_td", "rush_long", "fumbles", "fumbles_lost"
]

game_rec_stats = [
    "targets", "rec", "rec_yds", "rec_td", "rec_long", "fumbles", "fumbles_lost"
]

game_def_tackle_stats = [
    "sacks", "tackles_solo", "tackles_assists", "fumbles_forced", "fumbles_rec"
]

game_def_int_stats = [
    "def_int", "fumbles_forced", "fumbles_recs"
]

game_kicking_stats = [
    "xpm", "xpa", "fgm", "fga"
]

game_punting_stats = [
    "punt", "punt_yds_per_punt"
]

In [16]:
def get_player_indiv_game_stats(game_html, game_stats_list, side, player_link):
    
    indiv_player_stats_dict = {}
    if side in ["defense", "offense"]:
        game_stats_id = "player_{}".format(side)
    elif side in ["special_teams"]:
        game_stats_id = "kicking"
        
    game_stats_tbody = game_html.find("table", {"id" : game_stats_id}).tbody

    if game_stats_tbody.find("a", {"href" : player_link}) is not None:
        game_stats_row = game_stats_tbody.find("a", {"href" : player_link}).parent.parent
        player_stats = game_stats_row.findAll('td')    
        player_stats = list(filter(lambda x: x["data-stat"] in game_stats_list, player_stats))

        for td in player_stats:
            indiv_player_stats_dict[td["data-stat"]] = td.string 
    else:
        for stat in game_stats_list:
            indiv_player_stats_dict[stat] = 0
                
    return indiv_player_stats_dict

for tr in team_snaps_rows:
    player_link = tr.th.a['href']
    position = tr.find("td", {"data-stat" : "pos"}).string
    print(player_link, position)

    player = PlayerGame(player_link, position, team)
    num_snaps, perc_snaps = player.get_perc_snaps()
    if perc_snaps >= .10:
        print(perc_snaps)
        positional_player = player.player_class(player)
        pos_player_stats = positional_player.get_positional_stats(player)
        pos_player_stats["snaps"] = num_snaps
        print(pos_player_stats)

In [20]:
class PlayerGame():
    def __init__(self, player_link, position, Team):
        
        self.player_link = player_link   
        self.game_html = Team.game_html
        self.season = Team.season
        self.week = Team.week
        self.team = Team.team
        self.team_abbrev = Team.team_abbrev
        self.base_url = config['base_url']
        self.full_player_url = self.base_url + player_link
        self.standardized_position_dict = {
            "OL" : {"eligible_positions" : ["G", "T", "C", "LS"], 
                   "class": OffLineman,
                   "side" : "offense"},
            "QB" : {"eligible_positions" : ["QB"], 
                   "class": Quarterback,
                   "side" : "offense"},
            "WR" : {"eligible_positions" : ["WR"],
                   "class": WideReceiver,
                   "side" : "offense"},
            "TE" : {"eligible_positions" : ["TE"],
                   "class": TightEnd,
                   "side" : "offense"},
            "RB" : {"eligible_positions" : ["RB", "FB"],
                   "class": RunningBack,
                   "side" : "offense"},
            "DB" : {"eligible_positions" : ["SS", "FS", "CB", "DB"],
                   "class": DefBack,
                   "side" : "defense"},
            "LB" : {"eligible_positions" : ["LB", "OLB", "MLB"],
                   "class": Linebacker,
                   "side" : "defense"},
            "DL" : {"eligible_positions" : ["DT", "DL", "NT", "DE"],
                   "class": DefLineman,
                   "side" : "defense"},
            "K" : {"eligible_positions" : ["K"],
                   "class": Kicker,
                   "side" : "special_teams"},
            "P" : {"eligible_positions" : ["P"],
                   "class": Punter,
                   "side" : "special_teams"}
        }
        
        self.standardized_pos = [k for k, v in self.standardized_position_dict.items() \
                                 if position in v["eligible_positions"]][0]
        self.player_class = self.standardized_position_dict[self.standardized_pos]["class"]
        self.side = self.standardized_position_dict[self.standardized_pos]['side']
       # self.player_class = self.standardized_position_dict[self.standardized_pos]['class']
    
    def get_perc_snaps(self):
        
        if self.side == "offense":
            perc_snaps_id = "off_pct"
        elif self.side == "defense":
            perc_snaps_id = "def_pct"
        elif self.side == "special_teams":
            perc_snaps_id = "st_pct"
        perc_snaps_str = tr.find("td", {"data-stat" : perc_snaps_id}).string
        perc_snaps = float(int(perc_snaps_str.replace("%","")) / 100)
        
        num_snaps = tr.find("td", {"data-stat" : self.side}).string
        
        return num_snaps, perc_snaps
        
class OffLineman():
    def __init__(self, PlayerGame):
    
        self.id = "test"
        
    def get_positional_stats(self, PlayerGame):
        
        return {}

class Quarterback():
    def __init__(self, PlayerGame):
        
        self.game_tbl_passing_stats = game_pass_stats
        self.game_tbl_rushing_stats = game_rush_stats
    
    def get_positional_stats(self, PlayerGame):
        
        game_stats_list = game_pass_stats + game_rush_stats
        qb_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)
        
        return qb_stats
    
class RunningBack():
    def __init__(self, PlayerGame):
        
        self.game_tbl_rushing_stats = game_rush_stats
        self.game_tbl_rec_stats = game_rec_stats
        
    
    def get_positional_stats(self, PlayerGame):
        
        game_stats_list = self.game_tbl_rushing_stats + self.game_tbl_rec_stats
        rb_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return rb_stats

class WideReceiver():
    def __init__(self, PlayerGame):
        
        self.game_tbl_rushing_stats = game_rush_stats
        self.game_tbl_rec_stats = game_rec_stats
        
    
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_rushing_stats + self.game_tbl_rec_stats
        wr_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return wr_stats
    
class TightEnd():
    def __init__(self, PlayerGame):
        
        self.game_tbl_rec_stats = game_rec_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_rec_stats  
        te_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return te_stats

class DefLineman():

    def __init__(self, PlayerGame):
        
        self.game_tbl_def_stats = game_def_tackle_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_def_stats  
        dl_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return dl_stats
    
class Linebacker():

    def __init__(self, PlayerGame):
        
        self.game_tbl_def_stats = game_def_tackle_stats + game_def_int_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_def_stats  
        lb_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return lb_stats

class DefBack():
    
    def __init__(self, PlayerGame):
        
        self.game_tbl_def_stats = game_def_tackle_stats + game_def_int_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_def_stats  
        db_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return db_stats
    
class Kicker():
    
    def __init__(self, PlayerGame):
        
        self.game_tbl_k_stats = game_kicking_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_k_stats  
        k_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return k_stats
    
class Punter():
    
    def __init__(self, PlayerGame):
        
        self.game_tbl_p_stats = game_punting_stats
        
    def get_positional_stats(self, PlayerGame):

        game_stats_list = self.game_tbl_p_stats  
        p_stats = get_player_indiv_game_stats(PlayerGame.game_html, game_stats_list, PlayerGame.side, PlayerGame.player_link)

        return p_stats

In [21]:


    
#perc_snaps = tr.find("td", {"data-stat" : "off_pct"})

/players/H/HumpD.00.htm T
1.0
{'snaps': '68'}
/players/V/VeldJa20.htm T
1.0
{'snaps': '68'}
/players/S/ShipA.20.htm C
1.0
{'snaps': '68'}
/players/S/StanDr00.htm QB
1.0
{'pass_cmp': '11', 'pass_att': '28', 'pass_yds': '124', 'pass_int': '0', 'pass_sacked': '1', 'pass_rating': '77.1', 'rush_att': '2', 'rush_yds': '-2', 'rush_td': '0', 'rush_long': '-1', 'fumbles': '0', 'fumbles_lost': '0', 'snaps': '68'}
/players/B/BrowJo02.htm WR
0.94
{'rush_att': '0', 'rush_yds': '0', 'rush_td': '0', 'rush_long': '0', 'targets': '5', 'rec': '1', 'rec_yds': '11', 'rec_td': '0', 'rec_long': '11', 'fumbles': '1', 'fumbles_lost': '0', 'snaps': '64'}
/players/F/FitzLa00.htm WR
0.91
{'rush_att': '0', 'rush_yds': '0', 'rush_td': '0', 'rush_long': '0', 'targets': '8', 'rec': '6', 'rec_yds': '81', 'rec_td': '2', 'rec_long': '29', 'fumbles': '0', 'fumbles_lost': '0', 'snaps': '62'}
/players/G/GresJe00.htm TE
0.84
{'targets': '1', 'rec': '1', 'rec_yds': '4', 'rec_td': '0', 'rec_long': '4', 'fumbles': '0', 'fumbl

In [ ]:

# game_stats_tbody = game.game_html_page.find("table", {"id" : "kicking"}).tbody
# game_stats_tbody

# if game_stats_tbody.find("a", {"href" : player_link}) is not None:
#     game_stats_row = game_stats_tbody.find("a", {"href" : player_link}).parent.parent
#     player_stats = game_stats_row.findAll('td')    
#     player_stats = list(filter(lambda x: x["data-stat"] in game_stats_list, player_stats))

#     for td in player_stats:
#         indiv_player_stats_dict[td["data-stat"]] = td.string 
# else:
#     for stat in game_stats_list:
#         indiv_player_stats_dict[stat] = 0